## Plaque Classification Model Prediction

#### Developer: Meera Srikrishna

This notebook demonstrates how to load a pre-trained deep learning model, preprocess a set of images, and use the model to make predictions on the images. The images are classified into three categories: `coarse`, `cored`, and `diffused`. 

## Steps:
1. Import required libraries
2. Define paths to test images and model
3. Load and preprocess test images
4. Load the pre-trained model
5. Make predictions
6. Map numerical predictions to human-readable class labels
7. Display the results

#### 1. Import required libraries

In [19]:
import argparse
import os
import random
import shutil

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau

def create_model():
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=(120, 120, 3)),
        MaxPooling2D(2, 2),

        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),

        Flatten(),
        Dense(512, activation='relu'),
        Dense(3, activation='softmax')
    ])
    return model
)

IndexError: list index out of range

#### 2. Define paths to test images and model

In [2]:
test_dir = "./examples_inputs"
model_path = "cdc_rep_v1.h5"

#### 3. Load image filenames into a dataframe

In [5]:
test_img = os.listdir(test_dir)
test_df = pd.DataFrame({'Image': test_img})
test_df.head()

,Image
0,01_coarse_19.tif
1,20_cored_149.tif
2,55_diffused_1.tif


#### 4. Preprocess the images

In [6]:
test_gen = ImageDataGenerator(rescale=1./255)
# Set up the image generator for loading and preprocessing the images
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    test_dir, 
    x_col='Image',
    y_col=None,
    class_mode=None,
    target_size=(120, 120),
    batch_size=8,
    shuffle=False,
    color_mode='rgb'
)

Found 3 validated image filenames.


#### 5. Compile and load the trained model

In [12]:
model_pred = create_model()
model_pred.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
model_pred = load_model(model_path)

#### 6. Make predictions

In [18]:
# Predict on the test data
predictions = model_pred.predict(test_generator, steps=len(test_generator))
preds_cls_idx = predictions.argmax(axis=-1)

# Define class names
class_names = {
    0: "coarse",
    1: "cored",
    2: "diffused",
}
predicted_classes = [class_names[idx] for idx in preds_cls_idx]

# Display results
filenames_to_cls = list(zip(test_generator.filenames, predicted_classes))
for i, (filename, class_name) in enumerate(filenames_to_cls, start=1):
    print(f"Sample {i}: Filename: {filename}, Predicted Class: {class_name}")

UnimplementedError: Graph execution error:

Detected at node 'sequential_1/conv2d_4/Relu' defined at (most recent call last):
    File "C:\Users\meera\anaconda3\envs\plaque_classifier\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\meera\anaconda3\envs\plaque_classifier\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\meera\anaconda3\envs\plaque_classifier\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\meera\anaconda3\envs\plaque_classifier\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\meera\anaconda3\envs\plaque_classifier\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\meera\anaconda3\envs\plaque_classifier\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.

In [16]:
!pip show tensorflow


Name: tensorflow
Version: 2.13.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\meera\anaconda3\envs\plaque_classifier\lib\site-packages
Requires: tensorflow-intel
Required-by: 


In [17]:
!pip install --upgrade tensorflow


  Using cached tensorflow-2.13.1-cp38-cp38-win_amd64.whl.metadata (2.6 kB)
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.


In [20]:
import tensorflow as tf

# Check if TensorFlow can detect GPUs
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print(f"GPUs detected: {len(gpus)}")
    for i, gpu in enumerate(gpus):
        print(f"GPU {i}: {gpu.name}")
else:
    print("No GPUs detected")


No GPUs detected
